In [1]:
# import
import csv
import math
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline

random.seed

<bound method Random.seed of <random.Random object at 0x103080818>>

In [2]:
def IoW(gt, coor, radius_gt, radius):
    ### IoW cross section calculation
    cross_list_0 = [(gt[0]-radius_gt, 0), (gt[0]+radius_gt, 0)]
    cross_list_1 = [(gt[1]-radius_gt, 0), (gt[1]+radius_gt, 0)]
    
    ### IoW cross section calculation
    cross_list_0.append((coor[0]-radius, 1))
    cross_list_0.append((coor[0]+radius, 1))
    cross_list_1.append((coor[1]-radius, 1))
    cross_list_1.append((coor[1]+radius, 1))
                        
    cross_list_0.sort(key = lambda x : x[0])
    cross_list_1.sort(key = lambda x : x[0])

    if (cross_list_0[0][1] != cross_list_0[1][1] and cross_list_1[0][1] != cross_list_1[1][1]):
        return (cross_list_0[2][0] - cross_list_0[1][0]) * (cross_list_1[2][0] - cross_list_1[1][0]) / math.sqrt(radius*2)
    else:
        return 0.
    
    
class Net(nn.Module):
    def __init__(self, n_states, n_actions, n_hidden):
        super(Net, self).__init__()

        # 輸入層 (state) 到隱藏層，隱藏層到輸出層 (action)
        self.fc1 = nn.Linear(n_states, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden, n_actions)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x) # ReLU activation
        x = self.fc2(x)
        actions_value = self.out(x)
        return actions_value


class DQN(object):
    def __init__(self, n_states, n_actions, n_hidden, batch_size, lr, gamma, target_replace_iter, memory_capacity):
        self.eval_net, self.target_net = Net(n_states, n_actions, n_hidden), Net(n_states, n_actions, n_hidden)

        self.memory = np.zeros((memory_capacity, n_states * 2 + 2)) # 每個 memory 中的 experience 大小為 (state + next state + reward + action)
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=lr)
        self.loss_func = nn.MSELoss()
        self.memory_counter = 0
        self.learn_step_counter = 0 # 讓 target network 知道什麼時候要更新

        self.n_states = n_states
        self.n_actions = n_actions
        self.n_hidden = n_hidden
        self.batch_size = batch_size
        self.lr = lr
        self.gamma = gamma
        self.target_replace_iter = target_replace_iter
        self.memory_capacity = memory_capacity

    def choose_action(self, state, epsilon):
        x = torch.unsqueeze(torch.FloatTensor(state), 0)

        # epsilon-greedy
        if (np.random.uniform() < epsilon):
            action = np.random.randint(0, self.n_actions)
        else: 
            actions_value = self.eval_net(x) # 以現有 eval net 得出各個 action 的分數
            action = torch.max(actions_value, 1)[1].data.numpy()[0] # 挑選最高分的 action

        return action
    

    def store_transition(self, state, action, reward, next_state):
        # 打包 experience
        transition = np.hstack((state, [action, reward], next_state))

        # 存進 memory；舊 memory 可能會被覆蓋
        index = self.memory_counter % self.memory_capacity
        self.memory[index, :] = transition
        self.memory_counter += 1
        
        
    def learn(self):
        # 隨機取樣 batch_size 個 experience
        sample_index = np.random.choice(self.memory_capacity, self.batch_size)
        b_memory = self.memory[sample_index, :]
        b_state = torch.FloatTensor(b_memory[:, :self.n_states])
        b_action = torch.LongTensor(b_memory[:, self.n_states:self.n_states+1].astype(int))
        b_reward = torch.FloatTensor(b_memory[:, self.n_states+1:self.n_states+2])
        b_next_state = torch.FloatTensor(b_memory[:, -self.n_states:])

        # 計算現有 eval net 和 target net 得出 Q value 的落差
        q_eval = self.eval_net(b_state).gather(1, b_action) # 重新計算這些 experience 當下 eval net 所得出的 Q value
        q_next = self.target_net(b_next_state).detach() # detach 才不會訓練到 target net
        q_target = b_reward + self.gamma * q_next.max(1)[0].view(self.batch_size, 1) # 計算這些 experience 當下 target net 所得出的 Q value
        loss = self.loss_func(q_eval, q_target)

        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # 每隔一段時間 (target_replace_iter), 更新 target net，即複製 eval net 到 target net
        self.learn_step_counter += 1
        if self.learn_step_counter % self.target_replace_iter == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())

In [ ]:
# for floor in range(0,5):
#     for building in range(0,3):
for floor in range(2,3):
    for building in range(0,1):
        
        ## Count the number of data points in building id & floor id
        data_num = 0
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num += 1
        print(data_num)
        ## if there are no data, continue to next floor 
        if (data_num == 0):
            continue
            
        ## Load data points in
        wifi_loc_time = np.zeros(shape = (data_num, 524))
        i=-1
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i = i+1
                if (i > data_num):
                    break
                # wifi
                wifi_loc_time[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time[i-1][-1] = np.array(row[-1])
        
        ## Sort by time stamp
        wifi_loc_time = wifi_loc_time[wifi_loc_time[:,-1].argsort()]
        
        ## List all users
        user_list = np.unique(wifi_loc_time[:, 522])
        user_num = len(user_list)
        print("On floor ", floor, " in building ", building, "there are ", user_num, " users.")
        
                                    ## max boundary , grid size, min boundary
        grid_slice = 1
        longitude_list = np.array([max(wifi_loc_time[:, 520]), (max(wifi_loc_time[:, 520])-min(wifi_loc_time[:, 520]))/grid_slice\
                                   , min(wifi_loc_time[:, 520])])
        latitude_list = np.array([max(wifi_loc_time[:, 521]), (max(wifi_loc_time[:, 521])-min(wifi_loc_time[:, 521]))/grid_slice\
                                   , min(wifi_loc_time[:, 521])])
        
        ## set the sequence length of observed walking tracks
        seq_window = 9
        total_batch_num = 0
        
        ## Record each user's number of location logs
        user_walking_num = np.zeros(shape=(user_num), dtype=int)
        for i in range(len(wifi_loc_time[:,522])):
            for user_index in range(user_num):
                if (wifi_loc_time[i, 522] == user_list[user_index]):
                    user_walking_num[user_index] += 1
        
        ## Compute all users' walking sequences and add them as number of batches
        for num in user_walking_num:
            total_batch_num += math.floor(num/seq_window)
        print("total_batch_num: ",total_batch_num)
        
        batch_wifi = np.zeros((total_batch_num, seq_window, 520))
        batch_loc = np.zeros((total_batch_num, seq_window, 2), dtype = int)
        
        ## cumulative graduation [# of batch of user1] + [# of batch of user2] [...]
        batch_num = 0
        iter_batch = 0
        iter_window = 0
        
        for k in range(len(user_list)):
            print("User",k, " walking ", user_walking_num[k])
            
            ## [longitude, latitude, wifi]
            user_fp = np.zeros((user_walking_num[k], 2+520))
            batch_num += math.floor(user_walking_num[k]/seq_window)
            
            ## pick out the corresponding user's data
            for i in range(len(wifi_loc_time[:,522])):
                if (wifi_loc_time[i, 522] == user_list[k]):
                    batch_wifi[iter_batch][iter_window] = np.array(wifi_loc_time[i, :520])
                    batch_loc[iter_batch,iter_window] = np.array(wifi_loc_time[i, 520:522])

                    if (iter_window == (seq_window-1)):
                        iter_window = 0
                        iter_batch = iter_batch + 1
                        if (iter_batch == (batch_num)):
                            break
                    else:
                        iter_window = iter_window + 1
        ## Data preparation finishs
        ## batch_wifi, shape = (total_batch_num, seq_window, 520)
        ## batch_loc, shape = (total_batch_num, seq_window, 2)
        
        max_search_steps = 10
        ### DQN's hyper para
        #state: RSSI (520), coordinate (2), radius (1), history (50)
        n_actions = 5
        n_states = 520 + 2 + 1 + 50 
        n_hidden = 512
        batch_size = 100
        gamma = 0.1 # reward discount factor
        target_replace_iter = 100
        memory_capacity = 2000
        n_episodes = 1000
        lr = 1.
        eps = 1.
        
        ### Create DQN
        dqn = DQN(n_states, n_actions, n_hidden, batch_size, lr, gamma, target_replace_iter, memory_capacity)
        
        radius_gt = 0.5
        
        
        for k in range(n_episodes):
            print("Training k = ", k)
            for i in range(1):
                for j in range(1):
#             for i in range(total_batch_num):
#                 for j in range(seq_window):

                    # initial coordinates and radius
                    # radius_gt = target window
                    coordinate = np.array([(longitude_list[0]+longitude_list[2])/2. , (latitude_list[0]+latitude_list[2])/2.])
                    radius = max(longitude_list[0]-longitude_list[2], latitude_list[0]-latitude_list[2])/2. + radius_gt
                    
                    # initial history, 5n vector
                    history = np.zeros(shape=(5*max_search_steps,), dtype=int)
                    
                    # initial state: RSSI (520), coordinate (2), radius (1), history (50)
                    state = np.concatenate((batch_wifi[i][j], coordinate.copy(), np.array([radius]), history.copy()), axis=0)
                    
                    # Start a searching round
                    # eps
                    
                    alpha = 0.7
                    delta = 0.5
                    IoW_last_time = 0
                    Rewards = 0.
                    for t in range(max_search_steps):
                        # Select an action
                        action = dqn.choose_action(state, eps)

                        ## Plot gt region
                        if (k % 100 == 0 and i == 0 and j == 0 ):
                            fig = plt.figure()
                            ax = fig.add_subplot(111)
                            plt.xlim(longitude_list[0], longitude_list[2])
                            plt.ylim(latitude_list[0], latitude_list[2])
                            rect0 = plt.Rectangle((batch_loc[i,j,0]-radius_gt, batch_loc[i,j,1]-radius_gt), 2*radius_gt, 2*radius_gt, alpha=0.9)

                        #### To next state:
                        ### 1. New Center
                        ## 0 -> "Up Left"
                        ## 1 -> "Up Right"
                        ## 2 -> "Down Left"
                        ## 3 -> "Down Right"
                        ## 4 -> "Center"
                        if (action == 0):
                            coordinate[0] -= radius/2.
                            coordinate[1] += radius/2.
                        elif (action == 1):
                            coordinate[0] += radius/2.
                            coordinate[1] -= radius/2.
                        elif (action == 2):
                            coordinate[0] -= radius/2.
                            coordinate[1] -= radius/2.
                        elif (action == 3):
                            coordinate[0] += radius/2.
                            coordinate[1] -= radius/2.
                        
                        ### 2. New Radius
                        ## radius_t+1 = alpha * radius_t
                        radius *= alpha
                        
                        next_state = np.concatenate((batch_wifi[i][j], coordinate.copy(), np.array([radius]), history.copy()), axis=0)
                        
                        ### 3. Reward
                        IoW_transform = IoW(batch_loc[i,j], coordinate, radius_gt, radius)
                        
                        if (IoW_transform > delta):
                            reward = 3.0
                        elif (IoW_transform > IoW_last_time):
                            reward = 1.0
                        else:
                            reward = -3.0

                        # 儲存 experience
                        Rewards += reward
                        dqn.store_transition(state, action, Rewards, next_state)
                        
                        ### 4. History
                        one_hot = t*5 + action
                        history[one_hot] = 1
    
                        ## Plot searching region
                        if (k % 100 == 0 and i == 0 and j == 0 ):
                            print("K: ", k,"t: ", t, "action:", action)
                            rect1 = plt.Rectangle((coordinate[0]-radius, coordinate[1]-radius), 2*radius, 2*radius, alpha = 0.6)
                            ax.add_patch(rect0)
                            ax.add_patch(rect1)
                            plt.title("IoW_batch: "+str(k)+" loc: "+str(i)+"_"+str(j)+" round: "+str(t))
                            plt.savefig("IoW_batch_"+str(k)+"_loc_"+str(i)+"_"+str(j)+"_round_"+str(t))
                            plt.close()
                        
                        IoW_last_time = IoW_transform
                        
                        if dqn.memory_counter > memory_capacity:
                            dqn.learn()
                            
                        state = next_state
                        
            print('Episode finished after {}, {} location, total rewards {}'.format(i, j, Rewards))
                        
            eps = eps * 0.995

            
print("End! Bang!")       
                    


1443
On floor  2  in building  0 there are  2  users.
total_batch_num:  159
User 0  walking  733
User 1  walking  710
Training k =  0
K:  0 t:  0 action: 4
K:  0 t:  1 action: 4
K:  0 t:  2 action: 0
K:  0 t:  3 action: 3
K:  0 t:  4 action: 2
K:  0 t:  5 action: 2
K:  0 t:  6 action: 4
K:  0 t:  7 action: 4
K:  0 t:  8 action: 2
K:  0 t:  9 action: 4
Episode finished after 0, 0 location, total rewards -22.0
Training k =  1
Episode finished after 0, 0 location, total rewards -26.0
Training k =  2
Episode finished after 0, 0 location, total rewards -22.0
Training k =  3
Episode finished after 0, 0 location, total rewards -26.0
Training k =  4
Episode finished after 0, 0 location, total rewards -26.0
Training k =  5
Episode finished after 0, 0 location, total rewards -30.0
Training k =  6
Episode finished after 0, 0 location, total rewards -18.0
Training k =  7
Episode finished after 0, 0 location, total rewards -30.0
Training k =  8
Episode finished after 0, 0 location, total rewards -2

Episode finished after 0, 0 location, total rewards -22.0
Training k =  123
Episode finished after 0, 0 location, total rewards -30.0
Training k =  124
Episode finished after 0, 0 location, total rewards -26.0
Training k =  125
Episode finished after 0, 0 location, total rewards -22.0
Training k =  126
Episode finished after 0, 0 location, total rewards -30.0
Training k =  127
Episode finished after 0, 0 location, total rewards -26.0
Training k =  128
Episode finished after 0, 0 location, total rewards -26.0
Training k =  129
Episode finished after 0, 0 location, total rewards -26.0
Training k =  130
Episode finished after 0, 0 location, total rewards -30.0
Training k =  131
Episode finished after 0, 0 location, total rewards -30.0
Training k =  132
Episode finished after 0, 0 location, total rewards -30.0
Training k =  133
Episode finished after 0, 0 location, total rewards -30.0
Training k =  134
Episode finished after 0, 0 location, total rewards -26.0
Training k =  135
Episode fini

Episode finished after 0, 0 location, total rewards -30.0
Training k =  228
Episode finished after 0, 0 location, total rewards -30.0
Training k =  229
Episode finished after 0, 0 location, total rewards -30.0
Training k =  230
Episode finished after 0, 0 location, total rewards -26.0
Training k =  231
Episode finished after 0, 0 location, total rewards -30.0
Training k =  232
Episode finished after 0, 0 location, total rewards -22.0
Training k =  233
Episode finished after 0, 0 location, total rewards -30.0
Training k =  234
Episode finished after 0, 0 location, total rewards -30.0
Training k =  235
Episode finished after 0, 0 location, total rewards -22.0
Training k =  236
Episode finished after 0, 0 location, total rewards -30.0
Training k =  237
Episode finished after 0, 0 location, total rewards -22.0
Training k =  238
Episode finished after 0, 0 location, total rewards -26.0
Training k =  239
Episode finished after 0, 0 location, total rewards -26.0
Training k =  240
Episode fini

Episode finished after 0, 0 location, total rewards -30.0
Training k =  333
Episode finished after 0, 0 location, total rewards -30.0
Training k =  334
Episode finished after 0, 0 location, total rewards -30.0
Training k =  335
Episode finished after 0, 0 location, total rewards -30.0
Training k =  336
Episode finished after 0, 0 location, total rewards -30.0
Training k =  337
Episode finished after 0, 0 location, total rewards -30.0
Training k =  338
Episode finished after 0, 0 location, total rewards -30.0
Training k =  339
Episode finished after 0, 0 location, total rewards -30.0
Training k =  340
Episode finished after 0, 0 location, total rewards -26.0
Training k =  341
Episode finished after 0, 0 location, total rewards -30.0
Training k =  342
Episode finished after 0, 0 location, total rewards -30.0
Training k =  343
Episode finished after 0, 0 location, total rewards -30.0
Training k =  344
Episode finished after 0, 0 location, total rewards -30.0
Training k =  345
Episode fini

Episode finished after 0, 0 location, total rewards -30.0
Training k =  438
Episode finished after 0, 0 location, total rewards -30.0
Training k =  439
Episode finished after 0, 0 location, total rewards -30.0
Training k =  440
Episode finished after 0, 0 location, total rewards -30.0
Training k =  441
Episode finished after 0, 0 location, total rewards -30.0
Training k =  442
Episode finished after 0, 0 location, total rewards -30.0
Training k =  443
Episode finished after 0, 0 location, total rewards -30.0
Training k =  444
Episode finished after 0, 0 location, total rewards -30.0
Training k =  445
Episode finished after 0, 0 location, total rewards -30.0
Training k =  446
Episode finished after 0, 0 location, total rewards -30.0
Training k =  447
Episode finished after 0, 0 location, total rewards -30.0
Training k =  448
Episode finished after 0, 0 location, total rewards -30.0
Training k =  449
Episode finished after 0, 0 location, total rewards -30.0
Training k =  450
Episode fini

Episode finished after 0, 0 location, total rewards -26.0
Training k =  543
Episode finished after 0, 0 location, total rewards -26.0
Training k =  544
Episode finished after 0, 0 location, total rewards -22.0
Training k =  545
Episode finished after 0, 0 location, total rewards -26.0
Training k =  546
Episode finished after 0, 0 location, total rewards -26.0
Training k =  547
Episode finished after 0, 0 location, total rewards -26.0
Training k =  548
Episode finished after 0, 0 location, total rewards -26.0
Training k =  549
Episode finished after 0, 0 location, total rewards -26.0
Training k =  550
Episode finished after 0, 0 location, total rewards -26.0
Training k =  551
Episode finished after 0, 0 location, total rewards -26.0
Training k =  552
Episode finished after 0, 0 location, total rewards -26.0
Training k =  553
Episode finished after 0, 0 location, total rewards -26.0
Training k =  554
Episode finished after 0, 0 location, total rewards -26.0
Training k =  555
Episode fini

In [ ]:
#         # Testing
#         # extract data
#         print("Validation...")
#         data_num_v = 0
#         with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
#             spamreader_v = csv.reader(v_csvfile, delimiter=',')
#             for row in spamreader_v:
#                 if (row[523] == 'BUILDINGID'):
#                     continue
#                 elif (int(row[523]) is not building or int(row[522]) is not floor):
#                     continue
#                 data_num_v += 1

#         print(data_num_v)
#         if (data_num_v == 0):
#             continue
#         i = -1
#         wifi_loc_time_v = np.zeros(shape = (data_num_v, 524))
        
#         with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
#             spamreader_v = csv.reader(v_csvfile, delimiter=',')
#             for row in spamreader_v:
#                 if (row[523] == 'BUILDINGID'):
#                     continue
#                 elif (int(row[523]) is not building or int(row[522]) is not floor):
#                     continue
#                 i += 1
#                 #if (i > data_num):
#                 #    break
#                 # wifi
#                 wifi_loc_time_v[i-1][:520] = np.array(row[:520])
#                 # location x, y
#                 wifi_loc_time_v[i-1][520:522] = np.array(row[520:522])
#                 # userID
#                 wifi_loc_time_v[i-1][522] = np.array(row[526])
#                 # time stamp
#                 wifi_loc_time_v[i-1][-1] = np.array(row[-1])
                
#         wifi_loc_time_v = wifi_loc_time_v[wifi_loc_time_v[:,-1].argsort()]  
                
#         batch_num = int(data_num_v/seq_window)
#         print("Validating data ", batch_num)
#         batch_wifi_v = np.zeros((batch_num + 1, seq_window, 520))
#         batch_loc_v = np.zeros((batch_num + 1, seq_window), dtype = int)

#         iter_window = 0
#         iter_batch = 1
        
#         map_distance = 0
#         adjust_grid_num = 0

#         for s in range(len(wifi_loc_time_v)):
            
#             i = int(math.ceil((wifi_loc_time_v[s][520]-longitude_list[2])/longitude_list[1]) - 1)
#             j = int(math.ceil((wifi_loc_time_v[s][521]-latitude_list[2])/latitude_list[1]) - 1)
#             if (i < -1):
#                 i = 0
#             elif (i >= grid_slice): 
#                 i = grid_slice-1
#             if (j < 0):
#                 j = 0
#             elif (j >= grid_slice): 
#                 j = grid_slice-1
#             if (state_tag[i][j] == -1):
#                 adjust_grid_num+=1
#                 min_dist = np.zeros(shape=(state_count, 1))
#                 for x in range(state_count):
#                     min_dist[x,0] = math.sqrt(pow((float(state_map_grid[x][0])+0.5)*longitude_list[1]+longitude_list[2] - wifi_loc_time_v[s][520],2)+\
#                               pow((float(state_map_grid[x][1])+0.5)*latitude_list[1]+latitude_list[2] - wifi_loc_time_v[s][521],2))
#                 map_distance += np.min(min_dist)
#                 state_tag[i][j] = np.argmin(min_dist)
            
#             batch_wifi_v[iter_batch][iter_window] = wifi_loc_time_v[s][:520]
#             batch_loc_v[iter_batch,iter_window] = state_tag[i][j]
#             if (iter_window == (seq_window-1)):
#                 iter_window = 0
#                 iter_batch = iter_batch + 1
#                 if (iter_batch == (batch_num)):
#                     break
#             else:
#                 iter_window = iter_window + 1  
#         print("Grids need adjustment: ", adjust_grid_num)
#         print("Base map distance is ", map_distance/data_num_v)
                
#         model = ChainCRF()
#         ssvm = FrankWolfeSSVM(model=model, C=.1, max_iter=80)            
#         ssvm.fit(batch_wifi, batch_loc)
        
#         pred_batch_v = ssvm.predict(batch_wifi_v)
#         total_error = 0

#         center_longitude_pred = 0
#         center_latitude_pred = 0
#         center_longitude_gt = 0
#         center_latitude_gt = 0
        
#         real_longitude = 0
#         real_latitude = 0

#         iter_test = 0
        
#         window = np.zeros((seq_window))
        
#         gp_total_error = 0
#         gp_window = np.zeros((seq_window))
        
#         for i in range(1,batch_num+1):
#             for j in range(seq_window):
#                 ans = state_map_grid[pred_batch_v[i][j]][2].predict(batch_wifi_v[i][j].reshape(1,520))
                
#                 center_longitude_pred = (float(state_map_grid[pred_batch_v[i][j]][0])+0.5)*longitude_list[1]
#                 center_latitude_pred = (float(state_map_grid[pred_batch_v[i][j]][1])+0.5)*latitude_list[1]
#                 center_longitude_gt = (float(state_map_grid[batch_loc_v[i][j]][0])+0.5)*longitude_list[1]
#                 center_latitude_gt = (float(state_map_grid[batch_loc_v[i][j]][1])+0.5)*latitude_list[1]
                
                
                
                
#                 ans[0][0] += center_longitude_pred + longitude_list[2]
#                 ans[0][1] += center_latitude_pred + latitude_list[2]
#                 real_longitude = wifi_loc_time_v[iter_test][520]
#                 real_latitude = wifi_loc_time_v[iter_test][521]
                
#                 gp_total_error += math.sqrt(pow(ans[0][0] - real_longitude,2)+pow(ans[0][1] - real_latitude,2))
#                 gp_window[j] += math.sqrt(pow(ans[0][0] - real_longitude,2)+pow(ans[0][1] - real_latitude,2))
                
#                 iter_test+=1
                
#                 total_error += math.sqrt(pow(center_longitude_pred - center_longitude_gt,2)+pow(center_latitude_pred - center_latitude_gt,2))
                            
#                 window[j] += math.sqrt(pow(center_longitude_pred - center_longitude_gt,2)+pow(center_latitude_pred - center_latitude_gt,2))
                

#         print("\nbox\nRMSE: ", total_error/(batch_num*seq_window))
#         print("Windows:")
#         for x in range(seq_window):
#             print(" ", x, window[x]/batch_num)
#         print("gp")
#         print("RMSE: ", gp_total_error/(batch_num*seq_window))
#         print("Windows:")
#         for x in range(seq_window):
#             print(" ", x, gp_window[x]/batch_num)
#         print("Finish\n")